# Example: MLFlow integration

In this example we show how to use sQUlearn with the open-source MLOps tool [MLFlow](https://mlflow.org/docs/latest/index.html). The purpose of this tutorial is to show how sQUlearn integrates with tools that leverage the scikit-learn interface.

## Imports
We start by importing MLFlow as well as everything necessary to define a `QNNRegressor`.

In [1]:
import numpy as np
import mlflow

from squlearn import Executor
from squlearn.encoding_circuit import ChebyshevRx
from squlearn.observables import IsingHamiltonian
from squlearn.qnn import QNNRegressor, SquaredLoss
from squlearn.optimizers import SLSQP

## `QNNRegressor` Setup

Then we continue to set up a `QNNRegressor`. For more information refer to the [API docs](https://squlearn.github.io/modules/generated/squlearn.qnn.QNNRegressor.html).

In [2]:
nqubits = 4
number_of_layers = 2

reg = QNNRegressor(
    ChebyshevRx(nqubits, 1, num_layers=number_of_layers),
    IsingHamiltonian(nqubits, I="S", Z="S", ZZ="S"),
    Executor(),
    SquaredLoss(),
    SLSQP(),
)

## Helper Functions

We also need a helper function to extract the data collected by MLflow.

In [3]:
def fetch_logged_data(run_id):
    client = mlflow.MlflowClient()
    data = client.get_run(run_id).data
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, artifacts

## Training

Now we are ready to train the model to fit the logarithm funciton. We will keep track of the following things with the help of MLFlow:
- The `QNNRegressor`'s parameters
- The trained model state
- the models accuracy on the training set

In [4]:
x_space = np.arange(0.1, 0.9, 0.1).reshape(-1, 1)
ref_values = np.log(x_space).ravel()

with mlflow.start_run() as run:
    mlflow.log_params(reg.get_params())

    reg.fit(x_space, ref_values)
    mlflow.sklearn.log_model(reg, "model")

    accuracy = reg.score(x_space, ref_values)
    mlflow.log_metric("accuracy", accuracy)

fit:  55%|█████▌    | 55/100 [00:10<00:08,  5.01it/s]


2024/08/07 07:56:16 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


## Evaluation

Let's extract the tracked data and display it, starting with the `QNNRegressor`'s parameters.

In [5]:
params, metrics, artifacts = fetch_logged_data(run.info.run_id)
print(params)

{'alpha': '4.0', 'batch_size': 'None', 'caching': 'True', 'callback': 'pbar', 'closed': 'False', 'encoding_circuit': '<squlearn.encoding_circuit.circuit_library.chebyshev_rx.ChebyshevRx object at 0x0000015A4A2AD720>', 'epochs': 'None', 'executor': '<squlearn.util.executor.Executor object at 0x0000015AA02D77C0>', 'I': 'S', 'loss': '<squlearn.qnn.loss.SquaredLoss object at 0x0000015AA02D77F0>', 'nonlinearity': 'arccos', 'num_features': '1', 'num_layers': '2', 'num_qubits': '4', 'operator': "SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'IZZI', 'ZIIZ', 'ZIZI', 'ZZII'],\n              coeffs=[ParameterExpression(1.0*p[0]), ParameterExpression(1.0*p[1]),\n ParameterExpression(1.0*p[1]), ParameterExpression(1.0*p[1]),\n ParameterExpression(1.0*p[1]), ParameterExpression(1.0*p[2]),\n ParameterExpression(1.0*p[2]), ParameterExpression(1.0*p[2]),\n ParameterExpression(1.0*p[2]), ParameterExpression(1.0*p[2]),\n ParameterExpression(1.0*p[2])])", 'optimizer': '<squlearn.o

Let's also display the tracked metrics, i.e. the accuracy score.

In [6]:
print(metrics)

{'accuracy': 0.9999759983564892}


Finally, let's display the information about the stored model artifact.

In [7]:
print(artifacts)

['model/MLmodel', 'model/conda.yaml', 'model/model.pkl', 'model/python_env.yaml', 'model/requirements.txt']


The storage of the model artifact also allows us to reload the model at any given time and use it.

In [8]:
reg2 = mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/model")
reg2.predict(x_space)

array([-2.30217964, -1.61179577, -1.19923184, -0.92138725, -0.68878989,
       -0.5136204 , -0.35529528, -0.22364429])